# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Stored Procedures](#stored-procedures)**
3. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/common/schemas.ipynb"
    ]
}
```

# **TABLES**

---

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STATIC TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_POPULATION_RWZI]'))
CREATE TABLE [VWSSTATIC].[CBS_POPULATION_RWZI] (
    [ID] [INT] PRIMARY KEY IDENTITY(1,1),
    [DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE(),
    [RWZI_CODE] [INT] NULL,
    [RWZI_NAME] [VARCHAR](100) NULL,
    [START_DATE] [DATE] NULL,
    [END_DATE] [DATE] NULL,
    [INHABITANTS] [INT] NULL,
    [REGION_TYPE] [VARCHAR](100) NULL,
    [REGION_CODE] [VARCHAR](100) NULL,
    [REGION_NAME] [VARCHAR](100) NULL,
    [SHARE] [VARCHAR](100) NULL,
    [SHARE_VALUE] [DECIMAL](19,3) NULL,
    [DESCRIPTION] [VARCHAR](100) NULL
);
GO

-- 3) CREATE STATIC INDEX(ES).....
IF NOT EXISTS(SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCIX_DLI_CBS_POPULATION_RWZI')
CREATE NONCLUSTERED INDEX [NCIX_DLI_CBS_POPULATION_RWZI] ON [VWSSTATIC].[CBS_POPULATION_RWZI]
(
    [DATE_LAST_INSERTED] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY];
GO

# **STORED PROCEDURES**

---

### STATIC &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_CBS_POPULATION_RWZI_STATIC_TO_STATIC]
AS
BEGIN
    BEGIN TRANSACTION;

        WITH CTE
        AS (
            SELECT 
                [ID],
                [RWZI_CODE],
                [RWZI_NAME],
                CASE
                    -- FOR THE RWZI / REGION COMBINATIONS THAT START AT 1-1-2021, CHANGE START DATE TO 1-1-2020
                    WHEN
                        -- ONLY SELECT RWZI / REGION COMBINATIONS THAT START AT 1-1-2021
                        MIN([START_DATE]) OVER (PARTITION BY [RWZI_CODE],[REGION_CODE]) = CAST('2021-01-01' AS DATE)
                        -- FOR THE RWZI / REGION COMBINATION WHERE THE DATE IS ADJUSTED, ONLY ADJUST DATE 1-1-2021 TO 1-1-2020
                        AND MIN([START_DATE]) OVER (PARTITION BY [RWZI_CODE], [REGION_CODE], [START_DATE]) = CAST('2021-01-01' AS DATE)
                    THEN CAST('2020-01-01' AS DATE)
                    ELSE [START_DATE]
                END AS [START_DATE],
                CASE 
                    WHEN LEN([END_DATE]) = 0 THEN CAST('9999-12-31' AS DATE) 
                    ELSE CONVERT(DATE,[END_DATE],105) 
                END AS [END_DATE],
                [INHABITANTS],
                [REGION_TYPE],
                [REGION_CODE],
                [REGION_NAME],
                [SHARE]
            FROM [VWSSTATIC].[CBS_POPULATION_RWZI]
            WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[CBS_POPULATION_RWZI]) 
                AND UPPER([DESCRIPTION]) = 'DEFINITIEF'
                AND [RWZI_CODE] > 0
        )
        UPDATE target
        SET
            target.[SHARE_VALUE] = PARSE(REPLACE(source.[SHARE], '%', '') AS DECIMAL(19,3) USING 'NL-NL'),
            target.[START_DATE] = source.[START_DATE]
        FROM [VWSSTATIC].[CBS_POPULATION_RWZI] target
        INNER JOIN CTE AS source ON source.[ID] = target.[ID]
        WHERE target.[SHARE_VALUE] IS NULL

    COMMIT;
END
GO

# **DATATINO CONFIGURATIONS**

---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'CBS_POPULATION_RWZI',
        @workflow_alternative_name NVARCHAR(50),
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

SET @workflow_alternative_name = CONCAT(N'STATIC_', @workflow_name);

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_alternative_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_alternative_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_alternative_name,
    @description = @workflow_description,
    @schedule = '0 0 1 */3 *', -- AT 12:00 AM, ON DAY 1 OF THE MONTH, EVERY 3 MONTHS
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/STATIC/Aantal_inwoners_per_verzorgingsgebied.csv'
    WHEN 'acceptance' THEN 'datafiles/STATIC/Aantal_inwoners_per_verzorgingsgebied.csv'
    ELSE 'datafiles/STATIC/Aantal_inwoners_per_verzorgingsgebied.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STATIC......
SET @source_id = NULL;
SET @source_name = CONCAT('SOURCE_', @workflow_name);
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = CONCAT('VWSSTATIC.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'rwzi_code|rwzi_naam|startdatum|einddatum|inwoners|regio_type|regio_code|regio_naam|aandeel|toelichting',
    @target_columns = 'RWZI_CODE|RWZI_NAME|START_DATE|END_DATE|INHABITANTS|REGION_TYPE|REGION_CODE|REGION_NAME|SHARE|DESCRIPTION',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'SemiColon',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STATIC TO STATIC......
SET @source_id = NULL;
SET @source = CONCAT('dbo.SP_', @workflow_name, '_STATIC_TO_STATIC');
SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STATIC_TO_STATIC');
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM STATIC TO STATIC');

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STATIC......
SET @process_id = NULL;
SET @process_name = CONCAT('PROCESS_', @workflow_name);
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_alternative_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_alternative_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STATIC TO STATIC......
SET @process_id = NULL;
SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STATIC_TO_STATIC');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STATIC TO STATIC');
SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STATIC_TO_STATIC');

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_alternative_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_alternative_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STATIC TO STORED PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_id = NULL;
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STATIC_TO_STATIC');
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STATIC_TO_STATIC');
SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name);
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_alternative_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_alternative_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO